# Initial AI Agent PPO w/o
NN (working) MR2 Drag Disc

In [7]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [8]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 600
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = True
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 14,
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 0 is HS, 1 is 400m
  'carChoice' : 0, # 0 is MR2, 1 is Supra, 2 is Civic
  'rewardMode' : "simplex"
}

In [9]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [5]:
import ray
ray.shutdown()
ray.init()

2023-10-15 10:31:04,960	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.9
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [10]:
from ray.rllib.algorithms.ppo import PPOConfig
config = (
    PPOConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        #batch_mode="completed_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        train_batch_size=600,
        )
)

In [6]:
# import json

# class CustomEncoder(json.JSONEncoder):
#     def default(self, obj):
#         try:
#             json.dumps(obj)
#         except TypeError:
#             return "Not serializable object!"

#         return obj

# config_dict = config.to_dict()

# print(json.dumps(config_dict, sort_keys=True, indent=4, cls=CustomEncoder))

In [7]:
algo = config.build()

2023-10-14 22:11:44,430	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=10620) GT Real Time instantiated
(RolloutWorker pid=10620) GT AI Server instantiated for rtgym
(RolloutWorker pid=10620) starting up on localhost port 9999
(RolloutWorker pid=10620) Waiting for a connection
(RolloutWorker pid=10620) Connection from ('127.0.0.1', 54278)
(RolloutWorker pid=10620) J:\git\TensorFlowPSX\Py\dragSpaced.csv
(RolloutWorker pid=10620) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2


2023-10-14 22:11:54,840	INFO trainable.py:172 -- Trainable.setup took 10.412 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [8]:
# algo.get_policy().model

In [9]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()


(RolloutWorker pid=10620) reset triggered
(RolloutWorker pid=10620) Total reward was: 0
(RolloutWorker pid=10620) reload save for track : 9
(RolloutWorker pid=10620) reset triggered
(RolloutWorker pid=10620) Total reward was: 343.75
(RolloutWorker pid=10620) reload save for track : 9


(RolloutWorker pid=10620) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.17524280000017
(RolloutWorker pid=10620)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  0
Saved 0


(RolloutWorker pid=10620) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 1.2360348000000059
(RolloutWorker pid=10620)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10620) reset triggered
(RolloutWorker pid=10620) Total reward was: 200.625
(RolloutWorker pid=10620) reload save for track : 9


...

Loop:  888


(RolloutWorker pid=10620) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 1.14074160018572
(RolloutWorker pid=10620)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10620) reset triggered
(RolloutWorker pid=10620) Total reward was: 3213.25
(RolloutWorker pid=10620) reload save for track : 9


(RolloutWorker pid=10620) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.08655030004592845
(RolloutWorker pid=10620)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10620) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.14813480024895398
(RolloutWorker pid=10620)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=10620) reset triggered
(RolloutWorker pid=10620) Total reward was: 3635.5
(RolloutWorker pid=10620) reload save for track : 9


#Works!

In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-10-14_22-11-44ayp9l_kd/checkpoint_000881")

In [12]:
result = algo.train() #single try

(RolloutWorker pid=15780) reset triggered
(RolloutWorker pid=15780) Total reward was: 0
(RolloutWorker pid=15780) reload save for track : 9
(RolloutWorker pid=15780) reset triggered
(RolloutWorker pid=15780) Total reward was: 3628.625
(RolloutWorker pid=15780) reload save for track : 9


(RolloutWorker pid=15780) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.17203929999975642
(RolloutWorker pid=15780)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=15780) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.06114160000069546
(RolloutWorker pid=15780)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
